In [ ]:
DECLARE payoff_search_start TIMESTAMP DEFAULT TIMESTAMP("2024-07-01 00:00:00 UTC");
DECLARE payoff_search_end   TIMESTAMP DEFAULT TIMESTAMP("2025-07-30 00:00:00 UTC");
DECLARE usdc_addr STRING DEFAULT "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48";
DECLARE usdt_addr STRING DEFAULT "0xdac17f958d2ee523a2206206994597c13d831ec7";

WITH 
-- 1. LOAD DETECTED ATTEMPTS
attack_attempts AS (
    SELECT 
        attack_tx_hash,
        attack_ts,
        victim_address,
        attacker_address,
        token_symbol, 
        attack_type
    FROM `cs356-478822.eth_vlad_query.vald_query_attacks`
    WHERE attack_ts BETWEEN payoff_search_start AND TIMESTAMP("2025-07-01 00:00:00 UTC")
),

-- 2. FIND CANDIDATE PAYOFFS (Victim -> Attacker)
raw_payoffs AS (
    SELECT 
        transaction_hash AS payoff_tx_hash,
        block_timestamp AS payoff_ts,
        LOWER(from_address) AS victim_address,
        LOWER(to_address) AS attacker_address,
        SAFE_CAST(quantity AS BIGNUMERIC) / 1e6 AS payoff_value_usd,
        CASE WHEN LOWER(address) = usdc_addr THEN 'USDC' ELSE 'USDT' END AS token_symbol
    FROM bigquery-public-data.goog_blockchain_ethereum_mainnet_us.token_transfers
    WHERE 
        block_timestamp BETWEEN payoff_search_start AND payoff_search_end
        AND LOWER(address) IN (usdc_addr, usdt_addr)
        AND LOWER(from_address) IN (SELECT DISTINCT victim_address FROM attack_attempts)
        AND LOWER(to_address) IN (SELECT DISTINCT attacker_address FROM attack_attempts)
),

-- 3. CONFIRM THE ATTACK CHAIN
confirmed_payoffs AS (
    SELECT 
        p.payoff_tx_hash,
        p.payoff_ts,
        p.payoff_value_usd,
        p.token_symbol,
        a.attack_tx_hash,
        a.attack_ts,
        a.attack_type,
        p.victim_address, 
        TIMESTAMP_DIFF(p.payoff_ts, a.attack_ts, HOUR) AS hours_until_payoff
    FROM raw_payoffs p
    JOIN attack_attempts a
        ON p.victim_address = a.victim_address 
        AND p.attacker_address = a.attacker_address
    WHERE 
        p.payoff_ts > a.attack_ts
)

-- 4. FINAL REPORT
SELECT 
    payoff_tx_hash,
    payoff_ts,
    token_symbol,
    victim_address,
    attack_type,
    attack_tx_hash, -- This remains the single "most recent" attack hash
    
    -- NEW COLUMN: Collects ALL attack hashes for this payoff into an array
    ARRAY_AGG(attack_tx_hash) OVER(
        PARTITION BY payoff_tx_hash
    ) AS attack_attempts,

    payoff_value_usd,
    hours_until_payoff,
    CONCAT('https://etherscan.io/tx/', payoff_tx_hash) as etherscan_link
FROM confirmed_payoffs
WHERE payoff_value_usd > 10 

-- DEDUPLICATION
QUALIFY ROW_NUMBER() OVER(
    PARTITION BY payoff_tx_hash 
    ORDER BY attack_ts DESC
) = 1

ORDER BY payoff_value_usd DESC